# Modul 7 Praktikum Sains Data: Pengantar TensorFlow & Keras

Kembali ke [Sains Data](./saindat2024genap.qmd)

Sekarang kita sudah masuk ke materi *artificial neural network* (ANN) atau biasa disebut *neural network* (NN), yang mendasari dunia *deep learning*.

Saat modul praktikum ini disusun (April 2024), ada dua *framework* utama untuk *deep learning* di Python, yaitu:

1. TensorFlow: <https://www.tensorflow.org/>

    (dan Keras di dalamnya: <https://keras.io/>)

2. PyTorch: <https://pytorch.org/>

Kedua *framework* ini bersaing. Umumnya, TensorFlow lebih sering digunakan di industri, sedangkan PyTorch lebih sering digunakan dalam riset/penelitian.

Di pertemuan kali ini, kita akan membahas TensorFlow, baik penggunaannya secara sendiri (*pure TensorFlow*, yaitu tanpa Keras) maupun dengan bantuan Keras. Kalau belum punya, instal terlebih dahulu:

In [ ]:
pip install tensorflow

Keras terinstal bersama TensorFlow (karena Keras ada di dalamnya).

## Pengantar *Neural Network*

*Overview:*

- Secara umum, suatu *neural network* terdiri dari sejumlah *layer* atau lapisan (minimal dua).

- *Layer* pertama disebut *input layer*, dan *layer* terakhir disebut *output layer*.

- Tiap *layer* terdiri dari sejumlah *neuron*, yang masing-masing bisa menyimpan suatu nilai.

- Kecuali *input layer*, tiap *neuron* terhubung dengan sejumlah *neuron* di *layer* sebelumnya.

- Tiap sambungan terdiri dari nilai *weight* (sebagai pengali), nilai *bias* (sebagai pergeseran), dan suatu "fungsi aktivasi" yang menghasilkan nilai untuk *neuron* tujuan.

- *Weight* maupun *bias* disebut **parameter** dari *neural network*.

- Proses *training* adalah terus-menerus memperbarui parameter hingga hasil prediksi *neural network* sudah cukup baik, dengan meminimumkan suatu ***loss function*** atau fungsi objektif (yang intinya menghitung *error*).

- Suatu *neural network* bisa memiliki sejumlah *layer*, masing-masing dengan banyaknya *neuron* tertentu dan fungsi-fungsi aktivasi tertentu. Hal-hal itu disebut 
***hyperparameter*** dari *neural network*. Suatu **arsitektur** adalah suatu pilihan/konfigurasi *hyperparameter*.

### SLP: *(Single-Layer) Perceptron*

ANN paling pertama adalah *perceptron* (juga disebut SLP atau *single-layer perceptron*) yang dirancang oleh Frank Rosenblatt pada tahun 1957 (Géron, 2019). Ini adalah *neural network* yang paling sederhana, bahkan ini bisa disebut *building block* dari semua ANN (apabila diberi kebebasan untuk modifikasi). Konsep dasar *neural network* bisa kita pelajari di sini.

![](./gambar/aggarwal_nndl_hal5.png)

Sumber gambar: Aggarwal (2018) hal. 5

*Perceptron* hanya terdiri dari satu *input layer* dan satu *output layer*. Bahkan, aslinya hanya ada satu *neuron* di *output layer*.

Apabila dibutuhkan lebih dari satu *neuron* di *output layer*, itu bisa dianggap menggunakan lebih dari satu *perceptron* (yaitu menggunakan banyaknya *perceptron* sesuai banyaknya *neuron* di *output layer*), yang saling "ditumpuk":

![](./gambar/goodfellow_dl_hal337_crop.png)

Sumber gambar: Goodfellow, et. al. (2016) hal. 337

Aslinya, fungsi aktivasi yang digunakan oleh *perceptron* adalah *Heaviside step function* $H(v)$ yang mungkin kalian kenal dari mata kuliah PDB, atau juga disebut *threshold activation function*:

$$H(v) = \begin{cases}
    1, & v \ge 0 \\
    0, & v < 0
\end{cases}$$

Sehingga, untuk *output neuron* ke-$j$ yang disambung dari $n$ *input neuron*, model *perceptron* bisa dirumuskan sebagai berikut:

$$y_j = H\left(\left(\sum_{i=1}^{n} w_{ij} x_i \right) + b_j\right)$$

dengan

- $x_i$ adalah nilai pada *input neuron* ke-$i$

- $y_j$ adalah nilai pada *output neuron* ke-$j$

- $w_{ij}$ adalah parameter *weight* untuk sambungan *input neuron* ke-$i$ menuju *output neuron* ke-$j$

- $b_j$ adalah parameter *bias* untuk *output neuron* ke-$j$

Lebih umumnya,

$$y_j = \Phi\left(\left(\sum_{i=1}^{n} w_{ij} x_i \right) + b_j\right)$$

dengan $\Phi(v)$ adalah sembarang fungsi aktivasi.

*Note:* seperti di gambar, sebenarnya *bias* juga bisa dianggap *neuron* istimewa yang nilai $x_i$ nya selalu satu.

Biasanya, semua nilai di *layer* selanjutnya dihitung secara sekaligus menggunakan perkalian matriks, dengan perumusan:

$$\textbf{y} = \Phi\left(\textbf{w}^T \textbf{x} + \textbf{b}\right)$$

dengan $\textbf{x} = [x_i]$, $\textbf{y} = [y_j]$, dan $\textbf{b} = [b_j]$ adalah vektor kolom, serta $\textbf{w} = \left[w_{ij}\right]$ adalah matriks.

### MLP: *Multilayer Perceptron*

Konsep *single-layer perceptron* bisa diperumum menjadi *multilayer perceptron* atau *neural network* yang biasa kita kenal, dengan menambahkan beberapa *layer* di antara *input layer* dan *output layer*. Semua *layer* selain *input layer* dan *output layer* disebut *hidden layer*.

![](./gambar/aggarwal_nndl_hal18.png)

Sumber gambar: Aggarwal (2018) hal. 18

Konsep perhitungan antara tiap *layer* tetap sama, yaitu

$$\textbf{y} = \Phi\left(\textbf{w}^T \textbf{x} + \textbf{b}\right)$$

### Fungsi Aktivasi

![](./gambar/aggarwal_nndl_hal13.png)

Sumber gambar: Aggarwal (2018) hal. 13

Beberapa fungsi aktivasi adalah (Aggarwal, 2018, hal. 12-13):

a. "Linier" atau identitas

$$\Phi(v) = v$$

b. *Sign* (fungsi tanda): $\text{sign}(v)$ atau $\text{sgn}(v)$

$$
\Phi(v) = \text{sign}(v) = \begin{cases}
    1, & v > 0 \\
    0, & v = 0 \\
    -1, & v < 0
\end{cases}
$$

c. Sigmoid, terkadang dilambangkan $\sigma(v)$ dan terkadang disebut fungsi aktivasi logistik

$$\Phi(v) = \frac{1}{1 + e^{-v}}$$

d. *(Soft)* tanh: $\tanh(v)$

$$\Phi(v) = \frac{e^{2v} - 1}{e^{2v} + 1} = 2 * \text{sigmoid}(2v) - 1$$

e. *Rectified Linear Unit* (ReLU)

$$\Phi(v) = \max\{v, 0\}$$

f. *Hard tanh*

$$\Phi(v) = \max\{\min\{v, 1\}, -1\}$$

Fungsi aktivasi yang paling sering digunakan adalah ReLU, kecuali untuk *output layer*.

Untuk *output layer*, biasanya,

- untuk regresi, banyaknya *neuron* sesuai banyaknya nilai prediksi (umumnya hanya satu), dan digunakan fungsi aktivasi linier

- untuk klasifikasi *multiclass* (lebih dari dua kelas), biasanya banyaknya *output neuron* sesuai banyaknya kelas, dan digunakan fungsi aktivasi ***softmax*** sebagai berikut, agar *output* berupa peluang tiap kelas:

$$\Phi(\overline{v})_i = \frac{\exp(v_i)}{\sum_{j=1}^k \exp(v_j)}$$

- untuk klasifikasi biner, hanya ada satu *neuron* di *output layer*, dan digunakan fungsi aktivasi *sigmoid*. (Keberadaan hanya satu *output neuron* lebih hemat daripada menggunakan dua *output neuron*)

### *Loss function*

Misalkan $y_i$ adalah nilai sebenarnya dan $\hat{y}_i$ adalah hasil prediksi.

Untuk regresi, biasa digunakan MSE *(mean squared error)*, juga disebut *L2 loss*:

$$\text{MSE}(y, \hat{y}) = \frac{1}{n} \sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2$$

Untuk klasifikasi, biasa digunakan yang namanya *cross-entropy loss*, juga disebut *logistic loss* atau *log loss*:

$$L_{\text{log}}(y,\hat{y}) = -(y \ln (\hat{y}) + (1 - y) \ln (1 - \hat{y}))$$

### Proses *training*

Proses *training* untuk *neural network* dilakukan secara iteratif, yaitu tiap iterasi akan memperbarui parameter sehingga nilai *loss function* menjadi lebih kecil.

Tiap iterasi melakukan langkah-langkah berikut untuk tiap data *training*:

1. *Forward pass*: menghitung nilai *output* akhir, yaitu $\hat{y}$ (hasil prediksi), berdasarkan *input* data *training*.

2. Menghitung *loss* antara $y$ (nilai asli) dan $\hat{y}$

3. *Backpropagation*: menghitung gradien dari *loss* terhadap tiap parameter, secara "mundur"

4. *Update optimizer*: menggunakan algoritma *optimizer* seperti *gradient descent* untuk memperbarui parameter-parameter (*weights and biases*) berdasarkan gradien dari *loss*

    Note: ada banyak *optimizer*, seperti *gradient descent*, SGD (*stochastic gradient descent*), dan *Adam* (*adaptive moment estimation*). Pilihan *optimizer* (serta parameter-parameter yang bisa diatur untuk *optimizer*, seperti *learning rate*) juga menjadi *hyperparameter* untuk *neural network*.

Note: istilah *backward pass* meliputi langkah *backpropagation* dan *update optimizer*.

Apabila data *training* sangat banyak, terkadang data *training* tersebut dibagi menjadi beberapa *batch*, dan tiap iterasi menggunakan *batch* yang berbeda. Apabila semua *batch* sudah diproses, sebutannya adalah satu ***epoch***. Sehingga, satu *epoch* terdiri dari sejumlah iterasi sesuai banyaknya *batch*.

(Apabila data *training* tidak dibagi menjadi *batch*, maka satu *epoch* sama dengan satu iterasi.)

## *(Pure)* TensorFlow

In [ ]:
import tensorflow as tf

### Variabel dan Tensor

### *Automatic differentiation* dengan `GradientTape`

### Klasifikasi biner dengan *perceptron*

## Keras

In [ ]:
from tensorflow import keras

### *Sequential API*

### *Functional API*

### *Subclassing API* (yaitu dengan OOP)

## Referensi

Sumber gambar

- Aggarwal, C. Charu. 2018. *Neural Networks and Deep Learning: A Textbook.* Edisi Pertama. Springer.

- Goodfellow, Ian; Bengio, Yoshua; & Courville, Aaron. 2016. *Deep Learning*. MIT Press.

Buku lainnya

- Géron, Aurélien. 2019. *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems.* Edisi Kedua. O'Reilly Media.

Internet

- <https://www.tensorflow.org/api_docs/python/tf/GradientTape>

- <https://keras.io/api/models/>